In [ ]:
import numpy as np
import warnings
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator
warnings.simplefilter('ignore')

To load the hdmpy package which is used in the code below run the method below run from a jupyter notebook cell the code below
```
!pip install multiprocess
!pip install pyreadr
!git clone https://github.com/maxhuppertz/hdmpy.git
```

In [ ]:
import sys
sys.path.insert(1, "./hdmpy")
import hdmpy

In [ ]:
# An estimator class that runs the lasso with theoretically driven penalty choice.
# Better in small samples than cross-validation and also faster computationally
class RLasso(BaseEstimator):
    
    def __init__(self, *, post=False):
        self.post = post

    def fit(self, X, y):
        self.rlasso_ = hdmpy.rlasso(X, y, post=self.post)
        return self

    @property
    def coef_(self):
        return np.array(self.rlasso_.est['beta']).flatten()

    def predict(self, X):
        return X @ self.coef_ + np.array(self.rlasso_.est['intercept'])

In [ ]:
# A simple experimental data generating process. No effect heterogeneity.
def gen_data(n, d, p, delta, base):
    X = np.random.normal(0, 1, size=(n, d))
    D = np.random.binomial(1, p, size=(n,))
    y0 = base - X[:, 0] + np.random.normal(0, .1, size=(n,))
    y1 = delta + base - X[:, 0] + np.random.normal(0, .1, size=(n,))
    y = y1 * D + y0 * (1 - D)
    return y, D, X

In [ ]:
n = 100 # n samples
d = 100 # n features
delta = 1.0 # treatment effect
base = .3 # baseline outcome

In [ ]:
# Simple two means estimate and calcualtion of variance
def twomeans(y, D):
    hat0 = np.mean(y[D==0]) # mean of outcome of un-treated
    hat1 = np.mean(y[D==1]) # mean of outcome of treated
    V0 = np.var(y[D==0]) / np.mean(1 - D) # asymptotic variance of the mean of outcome of untreated
    V1 = np.var(y[D==1]) / np.mean(D) # asymptotic variance of the mean of outcome of treated
    hat = hat1 - hat0 # estimate of effect
    stderr = np.sqrt((V0 + V1) / n) # standard error of estimate of effect
    return hat, stderr

In [ ]:
np.random.seed(123)
y, D, X = gen_data(n, d, .2, delta, base) # generate RCT data
twomeans(y, D) # calculate estimation quantities

In [ ]:
from sklearn.linear_model import LinearRegression
# We implement the partialling out version of OLS (for pedagogical purposes)
def partialling_out(y, D, W):
    yres = y - LinearRegression().fit(W, y).predict(W) # residualize outcome with OLS
    Dres = D - LinearRegression().fit(W, D).predict(W) # residualize treatment with OLS
    hat = np.mean(yres * Dres) / np.mean(Dres**2) # calculate final residual ~ residual ols estimate
    epsilon = yres - hat * Dres # calculate residual of final regression; epsilon in the BLP decomposition
    V = np.mean(epsilon**2 * Dres**2) / np.mean(Dres**2)**2 # calculate variance of effect
    return hat, np.sqrt(V / y.shape[0]) # return estimate and standard error

In [ ]:
partialling_out(y, D, np.hstack([D*X, X]))

In [ ]:
# Now we simply replace OLS with Lasso to implement the Double Lasso process
def double_lasso(y, D, W):
    yres = y - RLasso().fit(W, y).predict(W) # residualize outcome with Lasso
    Dres = D - RLasso().fit(W, D).predict(W) # residualize treatment with Lasso
    # rest is the same as in the OLS case
    hat = np.mean(yres * Dres) / np.mean(Dres**2)
    epsilon = yres - hat * Dres
    V = np.mean(epsilon**2 * Dres**2) / np.mean(Dres**2)**2
    return hat, np.sqrt(V / y.shape[0])

In [ ]:
double_lasso(y, D, np.hstack([D*X, X]))

In [ ]:
# We now check the distributional properties of the different estimators across experiments
# First is the simple two means estimate
cov, hats, stderrs = [], [], [] # we will keep track of coverage (truth is in CI) and of the point estimate and stderr
for _ in range(100):
    y, D, X = gen_data(n, d, .2, delta, base)
    hat, stderr = twomeans(y, D)
    ci = [hat - 1.96 * stderr, hat + 1.96 * stderr] # 95% confidence interval
    hats += [hat]
    stderrs += [stderr]
    cov += [(ci[0] <= delta) & (delta <= ci[1])] # 1 if CI contains the true parameter

In [ ]:
np.mean(cov) # average coverage (should be .95 ideally)

In [ ]:
# distribution of estimates
plt.hist(hats)
plt.show()

In [ ]:
np.mean(hats) # mean of estimate; measures how biased the estimate is (should be =delta ideally)

In [ ]:
np.std(hats) # standard deviation of estimates; should be close to the standard errors we calculated for the CIs

In [ ]:
np.mean(stderrs)

In [ ]:
# Let's repeat this for the partialling out process (OLS), controlling for X
cov, hats, stderrs = [], [], [] # we will keep track of coverage (truth is in CI) and of the point estimate and stderr
for _ in range(100):
    y, D, X = gen_data(n, d, .2, delta, base)
    hat, stderr = partialling_out(y, D, X)
    ci = [hat - 1.96 * stderr, hat + 1.96 * stderr] # 95% confidence interval
    hats += [hat]
    stderrs += [stderr]
    cov += [(ci[0] <= delta) & (delta <= ci[1])] # 1 if CI contains the true parameter

In [ ]:
np.mean(cov)

In [ ]:
plt.hist(hats)
plt.show()

In [ ]:
np.mean(hats) # ols is heavily biased... mean of estimates very far from delta=1

In [ ]:
np.std(hats)

In [ ]:
np.mean(stderrs) # standard error severely under estimates the variance of the estimate; all this is due to overfitting

In [ ]:
# Now let's try the double Lasso. Because it's computationally expensive
# we'll do the experiments in parallel. Python makes parallelism very simple
from joblib import Parallel, delayed # we import these two functions

# we wrap our experiment process in a function, which is supposed to run a
# a single experiment
def exp(it, n, d):
    np.random.seed(it) # we draw a different seed for each experiment
    y, D, X = gen_data(n, d, .2, delta, base) # we generate data
    hat, stderr = double_lasso(y, D, X) # we apply the double lasso process
    ci = [hat - 1.96 * stderr, hat + 1.96 * stderr]
    # return estimate, standard error and (1 if CI contains the true parameter)
    return hat, stderr, (ci[0] <= delta) & (delta <= ci[1])

# Now here is how you run any function in parallel multiple times
# It's a simple parallel for loop.
res = Parallel(n_jobs=-1, verbose=3)(delayed(exp)(it, n, d) for it in range(100))

In [ ]:
# This simply takes the list of triples and turns it into a triple of lists :)
# good trick to know
hats, stderrs, cov = zip(*res)

In [ ]:
np.mean(cov) # much better coverage than OLS

In [ ]:
plt.hist(hats)
plt.show()

In [ ]:
np.mean(hats) # much closer to 1... (almost the same as two-means)

In [ ]:
np.std(hats) # standard deviation much smaller than two means, which did not adjust for X

In [ ]:
np.mean(stderrs) # and close to the calculate standard errors; we correctly estimated uncertainty

In [ ]:
# Let's see what would happen if we just run a single lasso
from joblib import Parallel, delayed

def exp(it, n, d):
    np.random.seed(it)
    y, D, X = gen_data(n, d, .2, delta, base)
    hat = RLasso().fit(np.hstack([D.reshape(-1, 1), X]), y).coef_[0]
    return hat # no obvious way to account for uncertainty

res = Parallel(n_jobs=-1, verbose=3)(delayed(exp)(it, n, d) for it in range(100))

In [ ]:
plt.hist(res)
plt.show()

In [ ]:
# bias is comparable and larger than standard deviation.
# Even if we could estimate the standard deviation, confidence intervals would undercover
1 - np.mean(res), np.std(res)

In [ ]:
# Let's try adding a post-Lasso OLS step and construct confidence
# intervals ignoring the Lasso step
from joblib import Parallel, delayed

def exp(it, n, d):
    np.random.seed(it)
    y, D, X = gen_data(n, d, .2, delta, base)
    # run a big lasso y ~ D, X
    DX = np.hstack([D.reshape(-1, 1), X])
    coefs = RLasso().fit(DX, y).coef_[1:]
    # run OLS on y ~ D, X[chosen by lasso]
    # calculate standard error as if lasso step never happened
    hat, stderr = partialling_out(y, D - np.mean(D), X[:, np.abs(coefs)>0.0])
    ci = [hat - 1.96 * stderr, hat + 1.96 * stderr]
    return hat, stderr, (ci[0] <= delta) & (delta <= ci[1])

res = Parallel(n_jobs=-1, verbose=3)(delayed(exp)(it, n, d) for it in range(100))

In [ ]:
hats, stderrs, cov = zip(*res)

In [ ]:
np.mean(cov) # not bad

In [ ]:
plt.hist(hats)
plt.show()

In [ ]:
1 - np.mean(hats), np.std(hats) # quite un-biased; bias < standard deviation

In [ ]:
# we under-estimated a bit the uncertainty; smaller estimated stderr than true std. 
# this is most prob a finite sample error, from ignoring the lasso variable selection step
# this is an RCT and so even post lasso ols is Neyman orthogonal. We should expect good behavior.
np.mean(stderrs)

In [ ]:
# But what if we are not in an RCT!?
import scipy.special
def gen_data(n, d, p, delta, base):
    X = np.random.normal(0, 1, size=(n, d))
    D = X[:, 0] + np.random.normal(0, 1/4, size=(n,))
    y = delta * D + base - X[:, 0] + np.random.normal(0, 1, size=(n,))
    return y, D, X

In [ ]:
from joblib import Parallel, delayed

def exp(it, n, d):
    np.random.seed(it)
    y, D, X = gen_data(n, d, .2, delta, base)
    DX = np.hstack([D.reshape(-1, 1), X])
    coefs = RLasso().fit(DX, y).coef_[1:]
    hat, stderr = partialling_out(y, D, X[:, np.abs(coefs)>0.0])
    ci = [hat - 1.96 * stderr, hat + 1.96 * stderr]
    return hat, stderr, (ci[0] <= delta) & (delta <= ci[1])

res = Parallel(n_jobs=-1, verbose=3)(delayed(exp)(it, n, d) for it in range(100))

In [ ]:
hats, stderrs, cov = zip(*res)

In [ ]:
np.mean(cov) # Oops! Post Lasso OLS severely undercovers; It is not Neyman orthogonal when D is correlated with X

In [ ]:
plt.hist(hats)
plt.show()

In [ ]:
np.mean(hats) # very heavily biased

In [ ]:
# But let's try the Neyman orthogonal Double Lasso
from joblib import Parallel, delayed

def exp(it, n, d):
    np.random.seed(it)
    y, D, X = gen_data(n, d, .2, delta, base)
    hat, stderr = double_lasso(y, D, X) # we apply the double lasso process
    ci = [hat - 1.96 * stderr, hat + 1.96 * stderr]
    return hat, stderr, (ci[0] <= delta) & (delta <= ci[1])

res = Parallel(n_jobs=-1, verbose=3)(delayed(exp)(it, n, d) for it in range(100))
hats, stderrs, cov = zip(*res)

In [ ]:
np.mean(cov) # great coverage

In [ ]:
1 - np.mean(hats), np.std(hats) # very small bias compared to standard deviation

In [ ]:
np.mean(stderrs) # accurate estimation of uncertainty

In [ ]:
# Approximately normal distribution of estimates, centered at the truth
plt.hist(hats)
plt.show()